<a href="https://colab.research.google.com/github/ria3999/mahine_learning-codes/blob/master/ChurnModellingUsingANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [18]:
df=pd.read_csv("/content/Churn_Modelling.csv")

In [19]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [20]:
X=df.iloc[:,3:13]
Y=df.iloc[:,13]

In [21]:
geography=pd.get_dummies(X["Geography"],drop_first=True)

In [22]:
gender=pd.get_dummies(X["Gender"],drop_first=True)

In [23]:
X=pd.concat([X,geography,gender],axis=1)

In [24]:
X=X.drop(['Geography','Gender'],axis=1)

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

In [27]:
from sklearn.preprocessing import StandardScaler

In [28]:
sc=StandardScaler()

In [29]:
X_train=sc.fit_transform(X_train)

In [30]:
X_test=sc.fit_transform(X_test)

We require feature scaling in deep learning.
w*x ko ek jaisa ho the convergence will be quick ...we will get global minima quickly.

In [31]:
import keras
from keras.models import Sequential
from keras.layers import Dense
#from keras.layers import LeakyRelu,PReLu,ELU
from keras.layers import Dropout

EMPTY NEURAL NETWORK

In [32]:
classifier=Sequential()

ADD LAYERS

In [33]:
classifier.add(Dense(units=6,kernel_initializer='he_uniform',activation='relu',input_dim=11))

In [34]:
classifier.add(Dense(units=6,kernel_initializer='he_uniform',activation='relu'))

usingg sigmoid to avoid vanishing gradient problem

In [35]:
classifier.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))

In [36]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


we should use binary crossentrophy if we have 2 categories in output and categorical crossentrophy if we have multiple categories in output

In [37]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [40]:
model=classifier.fit(X_train,Y_train,validation_split=0.33,batch_size=10,epochs=50)

Epoch 1/50
536/536 [==============================] - 1s 2ms/step - loss: 0.5713 - accuracy: 0.7380 - val_loss: 0.4903 - val_accuracy: 0.7929
Epoch 2/50
536/536 [==============================] - 1s 1ms/step - loss: 0.4592 - accuracy: 0.8039 - val_loss: 0.4515 - val_accuracy: 0.7982
Epoch 3/50
536/536 [==============================] - 1s 1ms/step - loss: 0.4287 - accuracy: 0.8169 - val_loss: 0.4339 - val_accuracy: 0.8076
Epoch 4/50
536/536 [==============================] - 1s 1ms/step - loss: 0.4121 - accuracy: 0.8244 - val_loss: 0.4174 - val_accuracy: 0.8156
Epoch 5/50
536/536 [==============================] - 1s 1ms/step - loss: 0.3995 - accuracy: 0.8289 - val_loss: 0.4045 - val_accuracy: 0.8213
Epoch 6/50
536/536 [==============================] - 1s 1ms/step - loss: 0.3887 - accuracy: 0.8324 - val_loss: 0.3976 - val_accuracy: 0.8236
Epoch 7/50
536/536 [==============================] - 1s 1ms/step - loss: 0.3804 - accuracy: 0.8378 - val_loss: 0.3910 - val_accuracy: 0.8307
Epoch 

In [41]:
y_pred=classifier.predict(X_test)

In [42]:
y_pred=(y_pred>0.5)

In [43]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,y_pred)

In [44]:
cm

array([[1541,   54],
       [ 221,  184]])

In [45]:
from sklearn.metrics import accuracy_score

In [46]:
score=accuracy_score(y_pred,Y_test)

In [47]:
score

0.8625

how to apply dropout

In [48]:
classifier.add(Dropout(0.3))

HYPERPARAMETER TUNING

In [49]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [50]:
from keras.models import Sequential

In [51]:
from keras.layers import Dense,Activation,Embedding,Flatten,BatchNormalization

In [52]:
from keras.activations import relu,sigmoid

In [53]:
def create_model(layers,activation):
  model=Sequential()
  for i,nodes in enumerate(layers):
    if i==0:
      model.add(Dense(nodes,input_dim=X_train.shape[1]))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
    else:
      model.add(Dense(nodes))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
    model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model


In [54]:
model=KerasClassifier(build_fn=create_model,verbose=0)

In [55]:
layers=[(20,),(40,20),(45,30,15)]
activations=['sigmoid','relu']
param_grid=dict(layers=layers,activation=activations,batch_size=[128,256],epochs=[30])
grid=GridSearchCV(estimator=model,param_grid=param_grid,cv=5)

In [56]:
grid_result=grid.fit(X_train,Y_train)

In [57]:
print(grid_result.best_score_,grid_result.best_params_)

0.853124988079071 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': (40, 20)}
